In [7]:
from google.cloud import bigquery
import os
import pandas as pd
import csv                                
import json
from google.cloud import pubsub_v1
import time

credentials_path = 'xxxxxxx.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials_path

publisher = pubsub_v1.PublisherClient()
topic_path = 'xxxxxxxxxx'

In [8]:
def rh_level(x):
    if x <= 30 : return 'Dry'
    elif x <=70: return 'Comfort'
    else: return 'Wet'

def us_aqi_level(x):
    if x <= 12 : return 'Good'
    elif x <=35.4: return 'Moderate'
    elif x <=55.4: return 'Unhealthy for Sensitive Groups '
    elif x <=105.4: return 'Unhealthy'
    elif x <=205.4: return 'Very Unhealthy'
    else: return 'Hazardous'

In [9]:
def read_file(path):
    with open(path, 'r') as f:          # Read lines separately
        read_line = f.readlines()
        read_line=read_line[1:]
    return read_line

def tranform(x):
    line=x.strip().split(",")
    line[0]=str(pd.to_datetime(x.strip().split(",")[0]))
    line[1]=float(line[1])
    line[2]=float(line[2])
    return line


In [10]:
lines=read_file('./data/air_quality.csv')
lines2=read_file('./data/temperature.csv')
lines_len=len(lines)

In [11]:
lines=read_file('./data/air_quality.csv')
lines2=read_file('./data/temperature.csv')
lines_len=len(lines)

for i in range(lines_len):
    
    lst=tranform(lines[i])
    lst2=tranform(lines2[i])

    df_temp_source1=pd.DataFrame({'time':[lst[0]],'pm2_5':[lst[1]],'uv_index':[lst[2]]})
    df_temp_source2=pd.DataFrame({'time':[lst2[0]],'temperature_2m':[lst2[1]],'relativehumidity_2m':[lst2[2]]})
    df=pd.merge(df_temp_source1,df_temp_source2,on='time')

    df['us_aqi_level']=df['pm2_5'].apply(lambda x:us_aqi_level(x))
    df['rh_level']=df['relativehumidity_2m'].apply(lambda x:rh_level(x))
    
    data_str = json.dumps(df.to_dict('records')[0])
    data = data_str.encode("utf-8")
    future = publisher.publish(topic_path, data)
    print(data)
    time.sleep(5)

b'{"time": "2023-01-04 00:00:00", "pm2_5": 14.0, "uv_index": 0.0, "temperature_2m": 22.7, "relativehumidity_2m": 63.0, "us_aqi_level": "Moderate", "rh_level": "Comfort"}'
b'{"time": "2023-01-04 01:00:00", "pm2_5": 17.0, "uv_index": 0.0, "temperature_2m": 23.4, "relativehumidity_2m": 63.0, "us_aqi_level": "Moderate", "rh_level": "Comfort"}'
b'{"time": "2023-01-04 02:00:00", "pm2_5": 18.0, "uv_index": 0.0, "temperature_2m": 24.8, "relativehumidity_2m": 57.0, "us_aqi_level": "Moderate", "rh_level": "Comfort"}'
b'{"time": "2023-01-04 03:00:00", "pm2_5": 14.0, "uv_index": 0.0, "temperature_2m": 26.0, "relativehumidity_2m": 54.0, "us_aqi_level": "Moderate", "rh_level": "Comfort"}'
b'{"time": "2023-01-04 04:00:00", "pm2_5": 11.0, "uv_index": 0.0, "temperature_2m": 27.2, "relativehumidity_2m": 51.0, "us_aqi_level": "Good", "rh_level": "Comfort"}'
b'{"time": "2023-01-04 05:00:00", "pm2_5": 10.0, "uv_index": 0.0, "temperature_2m": 28.6, "relativehumidity_2m": 49.0, "us_aqi_level": "Good", "rh_le